In [1]:
import glob
import polars as pl


In [2]:
E_VALUE_THRESHOLD = 0.001
suffix = "top_hits.tsv"

In [3]:
samples_dict = {"D1":("D1_NT", "D1_BH4"), "NoD1":("NoD1", "NoD1_BH4")}

spliced_trnas = "Arg-TCT Ile-TAT Leu-CAA Tyr-GTA Tyr-ATA".split()
black_list = [f"tRNA-{x}" for x in  spliced_trnas]
print(black_list)

cmscan_cols = [0, 2, 5, 6, 7, 8,  15]
col_names = ["trna_isotype", "query", "trna_isotype_start", 
             "trna_isotype_end", "query_start", "query_end", 
              "evalue"]


['tRNA-Arg-TCT', 'tRNA-Ile-TAT', 'tRNA-Leu-CAA', 'tRNA-Tyr-GTA', 'tRNA-Tyr-ATA']


In [4]:
cmscan_fn_list = glob.glob(f"./2023_07_19/*{suffix}")

counter = 0

for fn in cmscan_fn_list:
    sample_id = fn.split("/")[2].split(".")[0]
    #count_col_name = f"{sample_id}__count"
    print(fn, sample_id)
    tmp_df = pl.read_csv(fn, has_header=False, separator="\t", columns=cmscan_cols, new_columns = col_names ).filter(pl.col("trna_isotype").is_in(black_list).is_not()) 
    ### uncomment if needed
    ###.filter(pl.col("evalue") < E_VALUE_THRESHOLD)
    
    tmp_df = tmp_df.with_columns([pl.col("query").str.split(by="__").list.get(2).str.replace("count", "").cast(pl.Int32).alias("count"),
                                 pl.col('trna_isotype').str.replace("spld", "tRNA")]).sort(["trna_isotype", "trna_isotype_start", "trna_isotype_end"])
    df = tmp_df.select(["trna_isotype", "trna_isotype_start", "trna_isotype_end", "count"]).groupby(["trna_isotype", "trna_isotype_start", "trna_isotype_end"], maintain_order=True ).agg(pl.col("count").sum()) #.sort([]) 
    df = df.with_columns([pl.concat_str([pl.col("trna_isotype"), pl.col("trna_isotype_start"), pl.col("trna_isotype_end")], separator="_").alias("fragment_id"),
                         pl.lit(sample_id).alias("sample") ])


    #df.with_column(pl.lit(1).alias("constant_column"))
    #print(tmp_df.head(10))
    print(df.head(8))
    
    if counter == 0:
        result_df = df.select(pl.all())
    else:
        #xxx_df = df.select(["fragment_id", count_col_name])
        result_df = result_df.vstack(df)
    counter += 1
    print("XXX", result_df.head())

print(result_df.head())




./2023_07_19/NoD1_BH4.last.t2t_trnas.no_flank_reads.matches.top_hits.tsv NoD1_BH4
shape: (8, 6)
┌──────────────┬────────────────────┬──────────────────┬───────┬────────────────────┬──────────┐
│ trna_isotype ┆ trna_isotype_start ┆ trna_isotype_end ┆ count ┆ fragment_id        ┆ sample   │
│ ---          ┆ ---                ┆ ---              ┆ ---   ┆ ---                ┆ ---      │
│ str          ┆ i64                ┆ i64              ┆ i32   ┆ str                ┆ str      │
╞══════════════╪════════════════════╪══════════════════╪═══════╪════════════════════╪══════════╡
│ tRNA-Ala-AGC ┆ 43                 ┆ 73               ┆ 10    ┆ tRNA-Ala-AGC_43_73 ┆ NoD1_BH4 │
│ tRNA-Ala-AGC ┆ 45                 ┆ 73               ┆ 12    ┆ tRNA-Ala-AGC_45_73 ┆ NoD1_BH4 │
│ tRNA-Ala-AGC ┆ 46                 ┆ 73               ┆ 26    ┆ tRNA-Ala-AGC_46_73 ┆ NoD1_BH4 │
│ tRNA-Ala-AGC ┆ 47                 ┆ 73               ┆ 72    ┆ tRNA-Ala-AGC_47_73 ┆ NoD1_BH4 │
│ tRNA-Ala-CGC ┆ 1             

In [5]:
#NoD1_BH4	D1_BH4	NoD1_NT	D1_NT columns represent counts
combined_df = result_df.pivot(index =["fragment_id", "trna_isotype", "trna_isotype_start", "trna_isotype_end" ],  columns="sample", values="count", aggregate_function="first").with_columns(pl.col("NoD1_BH4	D1_BH4	NoD1_NT	D1_NT".split()).fill_null(strategy="zero")).sort("trna_isotype	trna_isotype_start	trna_isotype_end".split())
combined_df.write_csv("fragment_counts_combined_boseq_2023_07_21a.tsv", separator="\t")
combined_df

fragment_id,trna_isotype,trna_isotype_start,trna_isotype_end,NoD1_BH4,D1_BH4,NoD1_NT,D1_NT
str,str,i64,i64,i32,i32,i32,i32
"""tRNA-Ala-AGC_1…","""tRNA-Ala-AGC""",1,33,0,0,12,0
"""tRNA-Ala-AGC_8…","""tRNA-Ala-AGC""",8,53,0,0,0,10
"""tRNA-Ala-AGC_1…","""tRNA-Ala-AGC""",12,60,0,0,0,11
"""tRNA-Ala-AGC_4…","""tRNA-Ala-AGC""",43,73,10,118,0,0
"""tRNA-Ala-AGC_4…","""tRNA-Ala-AGC""",45,73,12,84,0,0
"""tRNA-Ala-AGC_4…","""tRNA-Ala-AGC""",46,73,26,131,0,0
"""tRNA-Ala-AGC_4…","""tRNA-Ala-AGC""",47,73,72,231,0,0
"""tRNA-Ala-CGC_1…","""tRNA-Ala-CGC""",1,26,0,10,20,0
"""tRNA-Ala-CGC_1…","""tRNA-Ala-CGC""",1,27,0,0,10,0


In [6]:
combined_df.describe()

describe,fragment_id,trna_isotype,trna_isotype_start,trna_isotype_end,NoD1_BH4,D1_BH4,NoD1_NT,D1_NT
str,str,str,f64,f64,f64,f64,f64,f64
"""count""","""1975""","""1975""",1975.0,1975.0,1975.0,1975.0,1975.0,1975.0
"""null_count""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,19.198987,55.667848,67.67443,346.476962,486.375696,261.996456
"""std""",null,null,16.215975,15.483005,548.006307,1975.996841,4102.999299,1842.10834
"""min""","""tRNA-Ala-AGC_1…","""tRNA-Ala-AGC""",1.0,25.0,0.0,0.0,0.0,0.0
"""max""","""tRNA_Tyr-GTA_9…","""tRNA_Tyr-GTA""",60.0,86.0,19037.0,44672.0,134773.0,43077.0
"""median""",null,null,15.0,56.0,0.0,10.0,14.0,14.0
"""25%""",null,null,4.0,43.0,0.0,0.0,0.0,0.0
"""75%""",null,null,35.0,71.0,0.0,45.0,59.0,39.0


In [7]:
isotype_end_df = combined_df.select(["trna_isotype", "trna_isotype_end"]).groupby("trna_isotype").max().sort("trna_isotype_end")
isotype_end_df


trna_isotype,trna_isotype_end
str,i64
"""tRNA-Ser-CGA""",47
"""tRNA-Val-AAC""",47
"""tRNA-Leu-TAA""",49
"""tRNA-Pro-CGG""",53
"""tRNA-Gln-TTG""",55
"""tRNA-Gly-GCC""",71
"""tRNA-Gly-CCC""",71
"""tRNA-Glu-TTC""",72
"""tRNA-Ala-TGC""",72
